## Postprocess and Analyze

In [ ]:
import logging
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.INFO)

In [ ]:
import pandas as pd
import src.simulation.plotting as plotting
import src.simulation.analysis as analysis
import src.simulation.postprocessing as postprocessing

test = False

do_postprocessing = False
if do_postprocessing:
    df = pd.read_csv("results/simulation_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    simulation_results = df.to_dict(orient="records")
    postprocessing_results = postprocessing.postprocess(simulation_results)
    pd.DataFrame(postprocessing_results).to_csv("results/postprocessing_results.csv")

do_analysis = True
if do_analysis:
    df = pd.read_csv("results/postprocessing_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    postprocessing_results = df.to_dict(orient="records")
    analysis_results = analysis.analyze(postprocessing_results)
    df = pd.DataFrame(analysis_results)
    df.to_csv("results/analysis_results.csv")

## Plot

In [ ]:
df = pd.read_csv("results/analysis_results.csv", index_col=0)
df = df[(df["std_lambda"] == 5.05e8) | (df["homogeneous"] == True)]
plotting.plot(df, create_subplots=True)

## Look at the cracks, the postprocessed data and the visual analysis results:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from scipy import ndimage

for index, row in df.iterrows():
    # Draw the original image
    # OP_01 = row["OP_01_bw"]
    # arr = np.load(OP_01)
    # plt.imshow(arr, cmap=plt.cm.gray)
    # plt.show()

    # Draw the skeleton
    skeleton = np.load(row["skeleton"])
    plt.imshow(skeleton, cmap=plt.cm.gray)
    plt.show()

    print('Skeleton length:', row["skeleton_length"])


In [ ]:
# TODO: Plot the fractures and check the results for different iso_values

## Plot Isolines

In [ ]:
#

In [ ]:
# from src.measure.isolines import isolines_from_vtk
# 
# import matplotlib.pyplot as plt
# import src.measure.curvature as curvature

# # random_indices = np.random.randint(0, len(df), 10)
# all_indices = np.arange(len(df))
# random_indices = all_indices

# y_min, y_max = -150, 150 #df["structured_mesh_min_y"].min(), df["structured_mesh_max_y"].max()

# for i in random_indices:
#     # Select row by using the column "run" which contains the run number which has the same range as the index
#     df_row = df.iloc[i]
#     isolines = isolines_from_vtk(
#         df_row["vtk_structured"],
#         df_row["iso_value"],
#     )
#     # plt.figure()
#     # curvatures = []
#     # for isoline in isolines:
#     #     isoline_curvature = curvature.calc_curvature(isoline, stride=20)
#     #     curvatures.append(isoline_curvature)

#     fig, ax1 = plt.subplots()
#     ax2 = ax1.twinx()
#     ax2.set_ylabel('Curvature', color='b')
#     ax1.set_ylabel('y')
#     ax1.set_ylim(y_min, y_max)

#     # Cutoff the isolines at x=900, because afterwards there is some artifact: The crack is widening (without reason, for most simulations)
#     cleaned_isolines = []
#     for isoline in isolines:
#         # remove all points with x > 900
#         indices = np.where(isoline[:,0] < 900)[0]
#         if len(indices) > 0:
#             isoline = isoline[indices]
#         cleaned_isolines.append(isoline)

#     isolines = cleaned_isolines

#     curvature_sum_sum = 0
#     for isoline in isolines:
#         if np.max(isoline[:, 0]) - np.min(isoline[:, 0]) < 100:
#             continue
#         isoline_curvature_local = curvature.calc_curvature(isoline, stride=20)
#         curvature_sum = np.sum(isoline_curvature_local)
#         curvature_sum_sum += curvature_sum
#         # x = np.linspace(0, 1000, len(isoline_curvature_local))
#         ax2.plot(isoline[:,0], isoline_curvature_local, color='b')
#         # plt.plot(isoline[:,0], isoline_curvature_local)
#         ax1.plot(isoline[:, 0], isoline[:, 1])
#     ax1.set_title(f"length={df_row['length']:.2f}, width={df_row['width']:.2f} volume={df_row['volume']:.2f}, curvature_sum={curvature_sum}") #, curvature={isoline_curvature:.2f}")
#     plt.show()

## Plot Volume